# Import packages

In [1]:
import pandas as pd
import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status
from dft_descriptors import prepocessing as pp
import logging
logging.basicConfig(level=logging.INFO)

# Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [2]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


# Provide smiles string(s)
and copy into them into my_smiles variable. It's a list of strings (a Python list can be of any size), the smiles are extracted for the csv resulting form the step 2 numbering the molecules.

In [6]:
react_nb = pd.read_csv("../data_csv/Data_test11032021.csv", sep=',')
react_nb = pp.preprocess(react_nb)

In [31]:
smiles = ['CC(C)C[Al](CC(C)C)c1ccc(F)cc1',
 'CC1(C)COB(c2cccc(C(F)(F)F)c2)OC1',
 'CCCCc1ccc(B2OB(c3ccc(CCCC)cc3)OB(c3ccc(CCCC)cc3)O2)cc1',
 'COc1cc(OC)cc([Al](CC(C)C)CC(C)C)c1',
 'COc1cccc(B(O)O)c1',
 'Cc1cc(C)c([Mg]Br)c(C)c1',
 'Cc1ccc([Mg]I)cc1',
 'Cc1cccc(C)c1[Al](CC(C)C)CC(C)C',
 'Cc1ccccc1B1OCC(C)(C)CO1',
 '[Li][Zn]([Li])(C)(C)(C)c1ccc(C(=O)N(C(C)C)C(C)C)cc1',
 '[Li]c1ccc(O[Si](C)(C)C(C)(C)C)cc1',
 'c1ccc([B-](c2ccccc2)(c2ccccc2)c2ccccc2)cc1']

# Initialize the molecules and generate conformations
We will use some for loops to loop over the smiles, only one conformation will be taken for each molecules in this work

In [32]:
mols = []
for smile in smiles:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC(C[Al](c1ccc(cc1)F)CC(C)C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC1(C)COB(OC1)c1cccc(c1)C(F)(F)F
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CCCCc1ccc(cc1)b1ob(ob(o1)c1ccc(cc1)CCCC)c1ccc(cc1)CCCC
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
*** Open Babel Error  in SMARTSError
  SMARTS Error:
o1[bH]o[bH]o[bH]1
   ^

INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1cc(OC)cc(c1)[Al](CC(C)C)CC(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:I

###### Draw some molecules

In [33]:
#mols[0].draw(0)

# Create gaussian job for each conformation

In [34]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C14H22AlF_ab46_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C12H14BF3O2_716d_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


...

C12H17BO2_04cd_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C12H19LiOSi_9440_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C24H20B-_85dd_conf_0


In [35]:
sm.get_job_stats(split_by_can=True)

status,created,done,failed,incomplete,submitted,uploaded
can,,,,,,
BrB(Br)Br,0,0,0,0,0,1
Br[Mg]C1CCCCC1,0,0,0,0,0,1
Br[Mg]Cc1ccccc1,0,0,0,0,0,1
Br[Mg]c1c(C)cc(cc1C)C,0,0,0,1,0,0
C(Oc1ccccc1)Oc1ccccc1,0,0,1,0,0,0
...,...,...,...,...,...,...
c1ccc(cc1)c1ccc2c(c1)c1cc(ccc1o2)c1ccccc1,0,0,0,0,1,0
c1ccc2c(c1)OCC2,0,0,0,0,1,0
c1ccc2c(c1)cc1c(c2)oc2c1cc1ccccc1c2,0,0,0,0,1,0


In [36]:
sm.submit_jobs()

INFO:aqc_utils.slurm_manager:Submitting 8 jobs.


Submitted batch job 10516


INFO:aqc_utils.slurm_manager:Submitted job 030f04201553bde44535b2a7f4ba59ec, job_id: 10516.


Submitted batch job 10517


INFO:aqc_utils.slurm_manager:Submitted job a5ce46004c4c2a827417bf168feceb32, job_id: 10517.


...

INFO:aqc_utils.slurm_manager:Submitted job 2cfd56c0df13d0866159d84e8a22b237, job_id: 10521.


Submitted batch job 10522


INFO:aqc_utils.slurm_manager:Submitted job ba5cea3b2d8de77db6fe5cc01146d17e, job_id: 10522.


Submitted batch job 10523


INFO:aqc_utils.slurm_manager:Submitted job a8975eeeddfee31f1e138984080a4968, job_id: 10523.


In [37]:
sm.squeue()

,RUNNING
jobs,19


In [92]:
for job in sm.get_jobs():
    if str(sm.get_jobs()[job].status) == 'slurm_status.done':
        print(sm.get_jobs()[job].base_name)

C20H28O2_1cb0_conf_0


In [93]:
job

'447581cd9c6bf05e9ba87c3a578b7d6d'

In [94]:
job_all = sm.get_jobs()[job]

In [95]:
sm.get_job_stats()

status,done,failed,incomplete,submitted,uploaded
jobs,1,343,3,5,93


In [87]:
str(job_all.status)

'slurm_status.created'

In [88]:
job_all.base_name

'C12H13NO2_b9f2_conf_0'